In [ ]:
# Q8
import cv2
import numpy as np
import matplotlib.pyplot as plt

def segment_and_blur_flower(image_path):
    """
    Q8: Use grabCut to segment flower and blur background
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not load image from {image_path}")
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # (a) Use grabCut for segmentation
    mask = np.zeros(img_rgb.shape[:2], np.uint8)
    
    # Initialize background and foreground models
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)
    
    # Define rectangle around the flower (adjust based on your image)
    height, width = img_rgb.shape[:2]
    rect = (width//6, height//6, width*2//3, height*2//3)
    
    # Apply grabCut
    cv2.grabCut(img_rgb, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)
    
    # Create final mask
    final_mask = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    
    # Extract foreground and background
    foreground = img_rgb * final_mask[:, :, np.newaxis]
    background = img_rgb * (1 - final_mask[:, :, np.newaxis])
    
    # (b) Blur background substantially
    blurred_background = cv2.GaussianBlur(background, (51, 51), 0)
    
    # Combine with original foreground
    enhanced_img = foreground + blurred_background
    
    return img_rgb, enhanced_img, final_mask, foreground, background, blurred_background

# Main execution
def main():
    image_path = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/daisy.jpg'
    
    try:
        # Apply segmentation and blur
        original, enhanced, mask, foreground, background, blurred_bg = segment_and_blur_flower(image_path)
        
        # Create comprehensive visualization
        plt.figure(figsize=(20, 12))
        
        # Row 1: Original and results
        plt.subplot(3, 4, 1)
        plt.imshow(original)
        plt.title('(a) Original Flower Image', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        plt.subplot(3, 4, 2)
        plt.imshow(mask, cmap='gray')
        plt.title('(a) Segmentation Mask', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        plt.subplot(3, 4, 3)
        plt.imshow(foreground)
        plt.title('(a) Foreground (Flower)', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        plt.subplot(3, 4, 4)
        plt.imshow(enhanced)
        plt.title('(b) Enhanced Image\n(Blurred Background)', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        # Row 2: Background processing
        plt.subplot(3, 4, 5)
        plt.imshow(background)
        plt.title('(a) Original Background', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        plt.subplot(3, 4, 6)
        plt.imshow(blurred_bg)
        plt.title('(b) Blurred Background\n(Gaussian 51x51)', fontweight='bold', fontsize=12)
        plt.axis('off')
        
        # Row 3: Histogram comparisons
        plt.subplot(3, 4, 7)
        plt.hist(original.ravel(), bins=50, alpha=0.5, color='blue', label='Original')
        plt.hist(enhanced.ravel(), bins=50, alpha=0.5, color='red', label='Enhanced')
        plt.title('Full Image Histogram Comparison', fontweight='bold', fontsize=12)
        plt.xlabel('Intensity')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(3, 4, 8)
        foreground_pixels = original[mask > 0]
        background_pixels = original[mask == 0]
        plt.hist(foreground_pixels.ravel(), bins=50, alpha=0.5, color='green', label='Foreground')
        plt.hist(background_pixels.ravel(), bins=50, alpha=0.5, color='purple', label='Background')
        plt.title('Foreground vs Background Histogram', fontweight='bold', fontsize=12)
        plt.xlabel('Intensity')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Edge analysis for question (c)
        plt.subplot(3, 4, 9)
        # Calculate gradient magnitude at edges
        gradient = cv2.Laplacian(enhanced, cv2.CV_64F)
        gradient_mag = np.abs(gradient).mean(axis=2)
        edge_mask = cv2.Canny(mask.astype(np.uint8)*255, 100, 200) > 0
        edge_gradients = gradient_mag[edge_mask]
        
        plt.hist(edge_gradients, bins=50, color='orange', alpha=0.7)
        plt.title('Edge Gradient Analysis', fontweight='bold', fontsize=12)
        plt.xlabel('Gradient Magnitude at Edges')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        
        # Difference visualization
        plt.subplot(3, 4, 10)
        diff = np.abs(original.astype(float) - enhanced.astype(float))
        plt.imshow(diff.mean(axis=2), cmap='hot')
        plt.title('Difference Map', fontweight='bold', fontsize=12)
        plt.axis('off')
        plt.colorbar()
        
        # Zoom on edge region for question (c)
        plt.subplot(3, 4, 11)
        # Extract a region around the edge
        y, x = np.where(mask > 0)
        if len(y) > 0:
            center_y, center_x = y.mean(), x.mean()
            roi_size = 100
            y_start = max(0, int(center_y) - roi_size//2)
            y_end = min(original.shape[0], y_start + roi_size)
            x_start = max(0, int(center_x) - roi_size//2)
            x_end = min(original.shape[1], x_start + roi_size)
            
            roi_original = original[y_start:y_end, x_start:x_end]
            roi_enhanced = enhanced[y_start:y_end, x_start:x_end]
            
            plt.imshow(np.hstack([roi_original, roi_enhanced]))
            plt.title('Edge Region Zoom (Original vs Enhanced)', fontweight='bold', fontsize=10)
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
        print("\nANSWERS TO QUESTION 8:")
        print("=" * 50)
        
        print("\n(a) grabCut Segmentation Results:")
        print("   - Successfully segmented flower from background")
        print("   - Created binary mask separating foreground/background")
        print("   - Extracted clean foreground and background components")
        
        print("\n(b) Background Blurring:")
        print("   - Applied Gaussian blur (51x51 kernel) to background")
        print("   - Combined blurred background with original foreground")
        print("   - Created professional-looking enhanced image")
        print("   - Flower remains sharp while background is artistically blurred")
        
        print("\n(c) Why background beyond flower edge is dark:")
        print("   - Gaussian blur spreads foreground pixels into background region")
        print("   - Segmentation mask imperfection causes mixed pixels at edges")
        print("   - Blending creates transition zone with mixed colors")
        print("   - This is a common artifact in segmentation-based blurring")
        print("   - The darkness is due to averaging of foreground and background colors")
        
        # Statistical analysis
        print("\n" + "=" * 60)
        print("STATISTICAL ANALYSIS")
        print("=" * 60)
        
        foreground_pixels = original[mask > 0]
        background_pixels = original[mask == 0]
        
        print(f"\n{'Metric':<20} {'Foreground':<12} {'Background':<12}")
        print("-" * 45)
        stats = [
            ('Mean Intensity', np.mean(foreground_pixels), np.mean(background_pixels)),
            ('Std Deviation', np.std(foreground_pixels), np.std(background_pixels)),
            ('Color Range', np.ptp(foreground_pixels), np.ptp(background_pixels))
        ]
        
        for metric, fg, bg in stats:
            print(f"{metric:<20} {fg:<12.2f} {bg:<12.2f}")
        
        print(f"\nSegmentation Quality:")
        print(f"Foreground area: {np.sum(mask > 0)} pixels")
        print(f"Background area: {np.sum(mask == 0)} pixels")
        print(f"Mask clarity: {np.mean(mask):.3f} (1.0 = perfect)")
        
        print(f"\nBlurring Effect:")
        print(f"Background blur strength: 51x51 Gaussian kernel")
        print(f"Sigma value: 0 (auto-calculated)")
        print(f"Visual effect: Professional portrait-style enhancement")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()